In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import torch  # type: ignore[import]
    import torch.nn as nn  # type: ignore[import]
    from torch.utils.data import TensorDataset, DataLoader, random_split  # type: ignore[import]
except ImportError as exc:
    raise ImportError(
        "PyTorch is required for this notebook. Please install torch."
    ) from exc
try:
    from sklearn.preprocessing import MinMaxScaler  # type: ignore[import]
except ImportError as exc:
    raise ImportError(
        "scikit-learn is required for this notebook. Please install scikit-learn."
    ) from exc

torch.manual_seed(42)

import sys
root_dir = os.path.abspath("..")
if root_dir not in sys.path:
    sys.path.append(root_dir)
# print("Project root in sys.path:", root_dir)

from src.config import (
    CMAPSS_DATASETS,
    MAX_RUL,
    SEQUENCE_LENGTH,
    HIDDEN_SIZE,
    NUM_LAYERS,
    OUTPUT_SIZE,
    LEARNING_RATE,
    NUM_EPOCHS
)
from src.data_loading import load_cmapps_subset, get_feature_drop_cols
from src.additional_features import create_physical_features
from src.training import train_and_evaluate_fd
from src.model import LSTMRULPredictor
from src.loss import rul_asymmetric_weighted_loss



In [2]:
fds = ["FD001", "FD002", "FD003", "FD004"]

all_results = {}
all_metrics = {}

for fd in fds:
    print("\n---------------------------------")
    print("Running local model for", fd)
    print("---------------------------------")
    res, m = train_and_evaluate_fd(
        fd_id=fd,
        model_class=LSTMRULPredictor,
        loss_fn=rul_asymmetric_weighted_loss,
    )
    all_results[fd] = res
    all_metrics[fd] = m


---------------------------------
Running local model for FD001
---------------------------------

=== Training subset FD001: 1 cond, 1 fault (HPC) ===
New columns successfully added. Current number of columns: 31
   Effizienz_HPC_Proxy  EGT_Drift  Fan_HPC_Ratio  RUL
0             0.941013        0.2       0.403737  125
1             0.943169        0.2       0.403406  125
2             0.942554       -1.8       0.404505  125
3             0.943025        0.2       0.405834  125
4             0.942581        1.2       0.405831  125
New columns successfully added. Current number of columns: 29
   Effizienz_HPC_Proxy  EGT_Drift  Fan_HPC_Ratio
0             0.941903        0.3       0.405617
1             0.941083        1.3       0.403985
2             0.941997        1.3       0.404842
3             0.941000       -0.7       0.405550
4             0.942237       -1.7       0.404810
Feature columns used for training:
['Setting1', 'Setting2', 'Sensor2', 'Sensor3', 'Sensor4', 'Sensor6', '

In [3]:
rows = []
for fd, m in all_metrics.items():
    rows.append({
        "FD": fd,
        "MSE": m["MSE"],
        "RMSE": m["RMSE"],
        "MAE": m["MAE"],
        "Bias": m["Bias"],
        "NASA": m["NASA_score"],
    })

metrics_df = pd.DataFrame(rows).set_index("FD")
metrics_df

,MSE,RMSE,MAE,Bias,NASA
FD,,,,,
FD001,304.096021,17.438349,13.289185,-7.877905,342.568248
FD002,371.025034,19.262010,15.032057,-5.547569,1713.732937
FD003,205.971389,14.351703,10.540734,-2.575181,278.030229
FD004,528.528124,22.989740,16.942756,-9.396808,2631.681699
